In [7]:
import torch
import torch.nn as nn
class myModel(nn.Module):
    def __init__(self, input_size, num_classes):
        super(myModel, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, 16),  # First dense layer
            nn.ReLU(),                  # Activation function
            nn.Dropout(0.2),            # Dropout for regularization
            nn.Linear(16, 5),         # Second dense layer
            # nn.ReLU(),                  # Activation function
            # nn.Dropout(0.2),            # Dropout for regularization
            # nn.Linear(64, num_classes)  # Output layer for classification
        )

    def forward(self, x):
        return self.model(x)


input_size = 187  
num_classes = 5  
model = myModel(input_size, num_classes)

state_dict = torch.load("myModel.pth",weights_only=True)

In [27]:
from optimum.quanto import quantize,qint2
import optimum.quanto as quanto

import pandas as pd
import optimum
quantize(model, weights=qint2, activations=qint2)
print(model.state_dict())
df2 =pd.read_csv('./Data/mitbih_test.csv')
input_size = 187 
num_classes = 5
Xtest,Ytest = df2.iloc[:, :-1],df2.iloc[:, -1]
Xtest_tensor = torch.tensor(Xtest.values, dtype=torch.float32)
Ytest_tensor = torch.tensor(Ytest.values, dtype=torch.long)


OrderedDict([('model.0.weight', tensor([[ 0.0404, -0.0366,  0.0250,  ...,  0.0108,  0.0537, -0.0697],
        [ 0.0244,  0.0567,  0.0565,  ...,  0.0364,  0.0147,  0.0252],
        [ 0.0026, -0.0020,  0.0314,  ..., -0.0125, -0.0697,  0.0646],
        ...,
        [-0.0611,  0.0165,  0.0121,  ...,  0.0340,  0.0200, -0.0092],
        [ 0.0180,  0.0592,  0.0730,  ..., -0.0546,  0.0208,  0.0490],
        [-0.0497, -0.0290,  0.0179,  ...,  0.0471, -0.0468,  0.0144]])), ('model.0.bias', tensor([ 0.0302,  0.0273, -0.0353, -0.0454, -0.0439, -0.0045, -0.0398,  0.0441,
         0.0542, -0.0731, -0.0338,  0.0660,  0.0209, -0.0179, -0.0216, -0.0521])), ('model.0.input_scale', tensor(1.)), ('model.0.output_scale', tensor(1.)), ('model.3.weight', tensor([[ 0.0359,  0.1900, -0.2431, -0.0708,  0.0843,  0.1009,  0.0708,  0.2271,
         -0.2390,  0.1863,  0.0530,  0.0728, -0.2228, -0.2030, -0.2399, -0.1589],
        [-0.0588,  0.0902,  0.1659, -0.1415,  0.2095, -0.1839,  0.2039, -0.0779,
         -0.17

In [28]:

model.eval()  
with torch.no_grad():  

    outputs = model(Xtest_tensor) 
    _, predicted = torch.max(outputs, 1)  

    correct = (predicted == Ytest_tensor).sum().item()
    total = Ytest_tensor.size(0)
    accuracy = correct / total * 100

print(f"Accuracy on test set: {accuracy:.2f}%")


ValueError: QBytesTensor can only be of 8-bit qtype